## Newcomb's Paradox

Suppose there are two boxes, A and B. You can choose to take the contents of either just box B, or both boxes A and B.
* You know that box A contains \$1000.
* Box B's contents were determined ahead of time by an omniscient adversary, who places:
    - \$1,000,000 if he predicts you will take Box B only
    - \$0 if he predicts you will take both boxes.

Assuming (because of determinism) that the adversary can indeed make such predictions perfectly, what should you do?

People have differing intuitions. Some think you should take both boxes and earn \$1,001,000. Others think you should only take box B.

Here, we model how Wolpert and Benford (2013) resolve the paradox. They argue that people's intuitions come from different intuitions of how to translate the English description into a formal game.

Wolpert, D. H., & Benford, G. (2013). The lesson of Newcomb’s paradox. Synthese, 190, 1637-1646.

We'll start with some groundwork…

In [1]:
from memo import memo
import jax
import jax.numpy as np
from enum import IntEnum

class Pick(IntEnum):
    # You could either take both A and B, or just B.
    AB = 0
    B = 1

# y is (y)our choice
# g is (g)od's choice
@jax.jit
def payout(g, y):
    return np.array([
        [1e3 + 000, 000],
        [1e3 + 1e6, 1e6]
    ])[g, y]

In Wolpert and Benford's words, the "fearful" interpretation is that you have "free will" to pick $y$ unconditionally, but God "knew" what you would have picked and predicts $g$ correctly. In this case, you should pick B only.

In [2]:
@memo
def fearful[p: Pick]():
    alice: chooses(y in Pick, to_maximize=imagine[
        god: knows(y),
        god: chooses(g in Pick, to_maximize=(g == y)),
        E[payout(god.g, y)]
    ])
    return Pr[alice.y == p]

for p, pr in zip(Pick, fearful()):
    print(f'Fearful chooses {p.name} with probability {pr}.')

Fearful chooses AB with probability 0.0.
Fearful chooses B with probability 1.0.


The "realist" interpretation is that God pre-registers $g$, and you have "free will" to pick the conditional distribution of $y \mid g$. In this case, you should pick both boxes.

In [3]:
@memo
def realist[p: Pick]():
    alice: thinks[ god: chooses(g in Pick, wpp=1) ]  # this distribution is arbitrary
    alice: chooses(y in Pick, to_maximize=E[payout(god.g, y)])
    return Pr[alice.y == p]

for p, pr in zip(Pick, realist()):
    print(f'Realist chooses {p.name} with probability {pr}.')

Realist chooses AB with probability 1.0.
Realist chooses B with probability 0.0.


## Advanced explorations

What should God do? It is tempting to implement a better version of the realist's model by having God choose $g$ by modeling Alice as choosing $y$ to be equal to $g$. But memo prohibits that: Alice can't choose based on a variable she doesn't yet know.

In [4]:
try:
    @memo
    def should_fail_to_compile[p: Pick]():
        alice: thinks[ god: chooses(g in Pick, wpp=imagine[
            alice: chooses(y in Pick, to_maximize=(g == y)),
            E[g == alice.y]
        ]) ]
        alice: chooses(y in Pick, to_maximize=(g == y))
        return Pr[alice.y == p]
except Exception as e:
    print(' Error:', e.message)
    print('\n'.join(e.__notes__))

 Error: Unknown choice alice.g

    at: @memo should_fail_to_compile in 810282331.py, line 5, column 52

  hint: Did you perhaps misspell g? alice is not yet aware of any
        choice called g. Or, did you forget to call alice.chooses(g
        ...) or alice.knows(g) earlier in the memo?

  ctxt: This error was encountered in the frame of alice, as modeled
        by god, as modeled by alice.  In that frame, alice is
        currently modeling the following 1 choices: y.

  P.S.: You are currently using...
        + memo version 0.3.1 and JAX version 0.4.30
        + on Python version 3.12.4 on the Darwin platform


Similarly, it is tempting to ask: what if Alice actually _does_ make the choice that God predicted? But memo catches the error here as well.

In [5]:
try:
    @memo
    def should_also_fail_to_compile[p: Pick]():
        alice: thinks[ god: chooses(g in Pick, wpp=1) ]
        alice: chooses(y in Pick, to_maximize=(y == god.g))
        return Pr[alice.y == p]
except Exception as e:
    print(' Error:', e.message)
    print('\n'.join(e.__notes__))

 Error: Uncertain about wpp

    at: @memo should_also_fail_to_compile in 3580391324.py, line 5, column 16

  hint: alice is uncertain about the value of the 'wpp' expression
        that alice is using to choose y. Hence, alice cannot compute
        the probabilities needed to make the choice. Perhaps you meant
        to take an expected value somewhere, using E[...]?

  ctxt: This error was encountered in the frame of alice.  In that
        frame, alice is currently modeling the following 2 choices:
        god.g, y.

  P.S.: You are currently using...
        + memo version 0.3.1 and JAX version 0.4.30
        + on Python version 3.12.4 on the Darwin platform
